# Searchlight

## How it works
Takes niftii images and applies a mask to generate spheres of inputed radius. Applies a function to each sphere and returns list of all spheres' results.

## Searchlight class
```
Implement search_light analysis using an arbitrary type of classifier.
    Parameters
    -----------
    mask_img : Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        boolean image giving location of voxels containing usable signals.
    func : function 
        to apply to each sphere
    args : list
        additional arguments for func
    process_mask_img : Niimg-like object, optional
        See http://nilearn.github.io/manipulating_images/input_output.html
        boolean image giving voxels on which searchlight should be
        computed.
    radius : float, optional
        radius of the searchlight ball, in millimeters. Defaults to 2.
    estimator : 'svr', 'svc', or an estimator object implementing 'fit'
        The object to use to fit the data
    %(n_jobs)s
    scoring : string or callable, optional
        The scoring strategy to use. See the scikit-learn documentation
        If callable, takes as arguments the fitted estimator, the
        test data (X_test) and the test target (y_test) if y is
        not None.
    cv : cross-validation generator, optional
        A cross-validation generator. If None, a 3-fold cross
        validation is used or 3-fold stratified cross-validation
        when y is supplied.
    %(verbose0)s
    Notes
    ------
    The searchlight [Kriegeskorte 06] is a widely used approach for the
    study of the fine-grained patterns of information in fMRI analysis.
    Its principle is relatively simple: a small group of neighboring
    features is extracted from the data, and the prediction function is
    instantiated on these features only. The resulting prediction
    accuracy is thus associated with all the features within the group,
    or only with the feature on the center. This yields a map of local
    fine-grained information, that can be used for assessing hypothesis
    on the local spatial layout of the neural code under investigation.
    Nikolaus Kriegeskorte, Rainer Goebel & Peter Bandettini.
    Information-based functional brain mapping.
    Proceedings of the National Academy of Sciences
    of the United States of America,
    vol. 103, no. 10, pages 3863-3868, March 2006
```

## Requirements for function


First two arguments must be X (spheres holding data) and the indices of the voxels in the sphere (use to work with other data not in sphere form, such as a noise regression). The rest are the additional arguments you pass to the searchlight class.

Example:  
```
def decode_cues(X, sphere_voxel_idxs, y, classifier, cues, runs, tent_length=9):
    ...


sl_obj = searchlight.SearchLight(frontal_mask, decode_cues, [subject_lss_data.trial_df.Cue.to_numpy(), clf, CUES, subject_lss_data.trial_df.Run.to_numpy()], verbose=1, radius=10.0, n_jobs=4)
```

## Example

In [1]:
from thalhi.decoding import SubjectLssTentData, decode_cues
from thalpy.decoding import searchlight
import pickle
from thalpy import base

THAL_HI_DIR = "/mnt/nfs/lss/lss_kahwang_hpc/data/ThalHi/"
CUES = ["dcb", "fcb", "dpb", "fpb", "dcr", "fcr", "dpr", "fpr"]

dir_tree = base.DirectoryTree(THAL_HI_DIR)
subjects = base.get_subjects(dir_tree.deconvolve_dir, dir_tree)

subject = next(sub for sub in subjects if sub.name == "10002")

os.chdir(subject.deconvolve_dir)
subject_lss_data = SubjectLssTentData.load("LSS_TENT.p")
subject_lss_data.remove_nan_trials()

clf = OneVsRestClassifier(LinearDiscriminantAnalysis())
frontal_mask = nib.load("/mnt/nfs/lss/lss_kahwang_hpc/ROIs/MNI_frontal.nii.gz")

img = nib.load(
    THAL_HI_DIR + "fmriprep/sub-10001/func/sub-10001_task-ThalHi_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
imgs = nib.Nifti1Image(subject_lss_data.data,
                        affine=img.affine, header=img.header)

# resample imgs and mask to 3x3x3
template = load_mni152_template(resolution=3)
frontal_mask = resample_to_img(frontal_mask, template, interpolation='nearest')
resampled_imgs = resample_to_img(imgs, template)

function = decode_cues
additional_args = [subject_lss_data.trial_df.Cue.to_numpy(), clf, CUES, subject_lss_data.trial_df.Run.to_numpy()]
sl_obj = searchlight.SearchLight(frontal_mask, function, additional_args, verbose=1, radius=10.0, n_jobs=4)
sl_obj.run(resampled_imgs)
print(sl_obj.output)
pickle.dump(sl_obj, open(os.path.join(dir_tree.deconvolve_dir, "searchlight_lda.p", "wb")), protocol=4)

Output will be list of whatever your function returns with a length equal to number of voxels in mask. It should then be easy to use the mask to transform data back to 3D brain space.  